In [37]:
import numpy as np
import os
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import sent_tokenize , word_tokenize
from natsort import natsorted
from numpy import dot
from numpy.linalg import norm
import string
import pandas as pd
import re
import nltk
import math
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [38]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [39]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [40]:
# Convert the string into lowercase to avoid case sensitive cases 
def convert_lowercase(data):
    return data.lower();

#separating words, keywords, phrases, symbols, and other items from a sequence of strings
def word_tokenization(data):
   word_tokens = word_tokenize(data)
   return word_tokens

#removing stop words ( common words in english that can be avoided or do not play a significant role in any sentence )
def remove_stopwords(data):
   stop_words = set(stopwords.words('english'))
   result = []
   for w in data:
      if w not in stop_words:
          result.append(w)
   return result

#removing punctuation marks from string
def remove_punctuationmarks(data):
    symbols = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    # symbols='''!()-[]{};:'"\,<>./?@#$%^&*_~=+'''
    for current in data:
      if current in symbols:
        data = data.replace(current, " ")
    return data

#remove blank space from string
def remove_blankspace(data):
  data = re.sub(' +', ' ', data)
  return data

# stemming to convert the given word into root word
def stemming(data):     
    stemmer= PorterStemmer()
    new_data = []
    for i in data:
        new_data.append(stemmer.stem(i))
    return new_data

# main calling function to call all the above pre processing steps
def preprocess(data): 
     data = remove_blankspace(data)
     data = remove_punctuationmarks(data)
     data = convert_lowercase(data)
     data = word_tokenization(data)
     data = remove_stopwords(data)
    #  data=stemming(data)
     return data

In [41]:
docMappingList = {}
d_docid = 0
d_tokenset = set()
tokenMapping = {}
revtokenMapping = {}
d_tid = 0

path = "/content/drive/MyDrive/Humor,Hist,Media,Food/"
fileList = os.listdir(path)

for i in fileList:
    file = open('/content/drive/MyDrive/Humor,Hist,Media,Food/'+i, 'r', encoding = "utf-8", errors ="ignore")
    text = file.read()
    preprocessed_tokens = preprocess(text)
    for x in preprocessed_tokens:
        if x not in d_tokenset :
          d_tokenset.add(x)
          tokenMapping[d_tid] = x
          revtokenMapping[x] = d_tid
          d_tid += 1 
    docMappingList[d_docid] = "/content/drive/MyDrive/Humor,Hist,Media,Food/" + i 
    d_docid += 1

d_tokenset = list(d_tokenset)

Generating TF-IDF

In [42]:
def gen_tfidf(tf, idf):
  rows = len(tf) 
  tf_idf = [[0 for i in range(d_tid)] for j in range(rows)]
  for i in range(rows):
    for j in range(d_tid):
      tf_idf[i][j] = tf[i][j] * idf[j] 
  return tf_idf

In [43]:
d_tf = [[0 for i in range(d_tid)] for j in range(d_docid)]
r = 0
for i in fileList :
      file = open('/content/drive/MyDrive/Humor,Hist,Media,Food/'+i, 'r', encoding = "utf-8", errors ="ignore")
      text = file.read()
      preprocessed_tokens = preprocess(text)
      for x in preprocessed_tokens:
          c = revtokenMapping[x]
          d_tf[r][c] = d_tf[r][c]+1
      r += 1
    
d_idf = []
for i in range(d_tid):
  c = 0
  for j in range(d_docid):
    if d_tf[j][i] > 0:
      c += 1
  d_idf.append(math.log(d_docid/(c+1)))

d_tfidf = gen_tfidf(d_tf, d_idf)

Query Vector TF

In [44]:
ip_tokens= []
str=input("Input sentence: ")
ip_tokens = preprocess(str)

ip_tf = [[0 for i in range(d_tid)] for j in range(1)]
ip_tfidf = [[0 for i in range(d_tid)] for j in range(1)]

r = 0
for i in ip_tokens :
    c = revtokenMapping[i]
    ip_tf[0][c] = ip_tf[0][c]+1

Input sentence: rinaldo help study


In [51]:
def gen_cos_sim(ip_tf, d_tf, choice):

  # BINARY FREQUENCY 
  if choice==1 :
    for i in range(d_docid):
      for j in range(d_tid):
        d_tf[i][j] = 0 if d_tf[i][j]==0 else 1 
    for i in range(d_tid):
      ip_tf[0][i] = 0 if ip_tf[0][i]==0 else 1 

  # TERM FREQUENCY
  elif choice==3 :
    for i in range(d_docid):
      sum_list = sum(d_tf[i]) 
      for j in range(d_tid):
        d_tf[i][j]  = d_tf[i][j]/sum_list
    sum_list = sum(ip_tf[0])
    for i in range(d_tid):
      ip_tf[0][i] = ip_tf[0][i]/sum_list

  # LOG NORMALIZATION
  elif choice==4 :
    for i in range(d_docid):
      for j in range(d_tid):
        d_tf[i][j]  = math.log2(1+d_tf[i][j])
    for i in range(d_tid):
      ip_tf[0][i] = math.log2(1+ip_tf[0][i])
   
  # DOUBLE NORMALIZATION 
  elif choice==5 :
    for i in range(d_docid):
      max_num = max(d_tf[i])
      for j in range(d_tid):
        d_tf[i][j]  = 0.5 + ( 0.5*( d_tf[i][j] / max_num ) )
    max_num = max(ip_tf[0])
    for i in range(d_tid):
      ip_tf[0][i] = 0.5 + ( 0.5*( ip_tf[0][i] / max_num ) )

  #  Calculating IDF for input and document using updated frequency matrix 
  for i in range(d_docid):
    for j in range(d_tid):
      d_tfidf[i][j] = d_tf[i][j] * d_idf[j] 
  
  for i in range(d_tid):
    ip_tfidf[0][i] = ip_tf[0][i] * d_idf[i]
     
  # Finding cosine similarity between tf-idf of query and documents one by one 
  COS = []
  for i in range(d_docid):
    cs = dot(ip_tfidf[0], d_tfidf[i] )/(norm(ip_tfidf[0])*norm(d_tfidf[i]))
    COS.append(cs)

  # Finding indexes of 5 maximum cosine similarity 
  max_5 = np.array(COS).argsort()[-5:][::-1]
  for i in range(5):
    print(docMappingList[max_5[i]])

BINARY FREQUENCY

In [46]:
gen_cos_sim(ip_tf, d_tf, 1)

/content/drive/MyDrive/Humor,Hist,Media,Food/rinaldos.txt
/content/drive/MyDrive/Humor,Hist,Media,Food/rinaldo.jok
/content/drive/MyDrive/Humor,Hist,Media,Food/rinaldos.law
/content/drive/MyDrive/Humor,Hist,Media,Food/resrch_phrase
/content/drive/MyDrive/Humor,Hist,Media,Food/resrch_p.hra


RAW COUNT

In [47]:
gen_cos_sim(ip_tf, d_tf,2)

/content/drive/MyDrive/Humor,Hist,Media,Food/rinaldos.txt
/content/drive/MyDrive/Humor,Hist,Media,Food/rinaldo.jok
/content/drive/MyDrive/Humor,Hist,Media,Food/rinaldos.law
/content/drive/MyDrive/Humor,Hist,Media,Food/resrch_phrase
/content/drive/MyDrive/Humor,Hist,Media,Food/resrch_p.hra


TERM FREQUENCY

In [48]:
gen_cos_sim(ip_tf, d_tf,3)

/content/drive/MyDrive/Humor,Hist,Media,Food/rinaldos.txt
/content/drive/MyDrive/Humor,Hist,Media,Food/rinaldo.jok
/content/drive/MyDrive/Humor,Hist,Media,Food/rinaldos.law
/content/drive/MyDrive/Humor,Hist,Media,Food/resrch_phrase
/content/drive/MyDrive/Humor,Hist,Media,Food/resrch_p.hra


LOG NORMALIZATION

In [49]:
gen_cos_sim(ip_tf, d_tf,4)

/content/drive/MyDrive/Humor,Hist,Media,Food/rinaldos.txt
/content/drive/MyDrive/Humor,Hist,Media,Food/rinaldo.jok
/content/drive/MyDrive/Humor,Hist,Media,Food/rinaldos.law
/content/drive/MyDrive/Humor,Hist,Media,Food/resrch_phrase
/content/drive/MyDrive/Humor,Hist,Media,Food/resrch_p.hra


DOUBLE NORMALIZATION

In [50]:
gen_cos_sim(ip_tf, d_tf,5)

/content/drive/MyDrive/Humor,Hist,Media,Food/boston.geog
/content/drive/MyDrive/Humor,Hist,Media,Food/flowchrt
/content/drive/MyDrive/Humor,Hist,Media,Food/justify
/content/drive/MyDrive/Humor,Hist,Media,Food/flowchrt.txt
/content/drive/MyDrive/Humor,Hist,Media,Food/banana05.brd
